In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

desired_frames = 1
desired_features = 2048

In [2]:
labels = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


In [4]:
scenes = list(pd.unique(labels["SCENE"]))

In [5]:
len(scenes)

1202

In [6]:
parent_dir = "/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/features"
visual_features = {}
context_visual_features = {}
for i in range(len(scenes)):
    # Visual Features
    try:
        vf = np.load(os.path.join(parent_dir, "utterances_final", "resnet_pool5_" + scenes[i] + ".npy"))
        #Global average pooling
        vf_p = np.apply_over_axes(np.mean, vf, [2, 3]) 
        vf_p = np.reshape(vf_p, (vf_p.shape[0],2048))
    except:
        vf_p = np.zeros((15, 2048))
        
    # Visual Features - Context
    try:
        c_vf = np.load(os.path.join(parent_dir, "context_final", "resnet_pool5_" + scenes[i] + "_c.npy"))
        #Global average pooling
        c_vf_p = np.apply_over_axes(np.mean, c_vf, [2, 3]) 
        c_vf_p = np.reshape(c_vf_p, (c_vf_p.shape[0],2048))
    except:
        c_vf_p = np.zeros((15, 2048))
    
    visual_features[scenes[i]] = vf_p
    context_visual_features[scenes[i]] = c_vf_p

In [7]:
def get_model_data(context_video_features, video_features):
    model_data = pd.DataFrame(columns=['context_video_feature', 'video_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
#         audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'context_video_feature': context_video_features[row["SCENE"]],
                                    'video_feature': video_features[row["SCENE"]],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [8]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data
   

In [9]:
class FNNTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'averaged_video_feature']
            ctxt_features = self.data.loc[index, 'averaged_context_video_feature']
            a=np.empty((1,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((ctxt_features, features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'averaged_video_feature']
            ctxt_features = self.data.loc[index, 'averaged_context_video_feature']
            final_features = np.hstack((ctxt_features, features))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
    
    def __getindexlist__(self):
        return list(self.data.index.values)

class FNNNetSID(nn.Module):
    def __init__(self):
        super(FNNNetSID, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(desired_frames*desired_features*2, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)

    def forward(self, x):
        x = x.view(-1, desired_frames*desired_features*2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
class FNNNetSD(nn.Module):
    def __init__(self):
        super(FNNNetSD, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(desired_frames*(desired_features + desired_features + 1), hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 2)

    def forward(self, x):
        x = x.view(-1, desired_frames*(desired_features + desired_features + 1))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
def predict_fnn(fnn_model, dataloader):
    prediction_list = []
    actual_list = []
    for data, target in dataloader:
        outputs = fnn_model(data)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
        actual_list.append(target)
    return prediction_list, actual_list

In [10]:
warnings.filterwarnings("ignore")

model_data = get_model_data(context_visual_features, visual_features)
# Label Encode Speaker
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,context_video_feature,video_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[0.30626756, 0.66872513, 0.28155187, 0.312841...","[[0.13835047, 0.2704592, 0.44648886, 0.1415337...",0.0,NONE,SHELDON,25
1,"[[0.17222668, 0.7350801, 0.30408216, 0.3297085...","[[0.46479157, 0.1813915, 0.22123067, 0.5245148...",0.0,NONE,PENNY,15
2,"[[0.048892517, 0.23698963, 0.4193368, 0.341263...","[[0.253619, 0.25664786, 0.6646118, 0.4821793, ...",0.0,NONE,RAJ,21
3,"[[0.5546928, 0.36914897, 0.8243424, 0.18547274...","[[0.55624646, 0.16990338, 0.62457716, 0.209021...",1.0,PRO,HOWARD,7
4,"[[0.73847526, 1.0405933, 0.11508263, 0.9313859...","[[0.6140023, 0.4846397, 0.79425097, 0.13518682...",0.0,NONE,SHELDON,25


In [11]:
train_data, test_data = get_train_test_split(
    model_data, ['context_video_feature', 'video_feature', 'speaker_encode'], 'sarcasm', 'sarcasm_type')
fnn_train = train_data.copy()
fnn_test = test_data.copy()
fnn_train.reset_index(drop=True, inplace = True)
fnn_test.reset_index(drop=True, inplace = True)

        
fnn_train['averaged_video_feature'] = fnn_train.loc[:, 'video_feature']
fnn_train['averaged_context_video_feature'] = fnn_train.loc[:, 'context_video_feature']
for index, row in fnn_train.iterrows():
    video = row['averaged_video_feature']
    fnn_train.at[index, "averaged_video_feature"] = np.array([np.mean(video, axis=0)])
    context_video = row['averaged_context_video_feature']
    fnn_train.at[index, "averaged_context_video_feature"] = np.array([np.mean(context_video, axis=0)])

fnn_test['averaged_video_feature'] = fnn_test.loc[:, 'video_feature']
fnn_test['averaged_context_video_feature'] = fnn_test.loc[:, 'context_video_feature']
for index, row in fnn_test.iterrows():
    video = row['averaged_video_feature']
    fnn_test.at[index, "averaged_video_feature"] = np.array([np.mean(video, axis=0)])
    context_video = row['averaged_context_video_feature']
    fnn_test.at[index, "averaged_context_video_feature"] = np.array([np.mean(context_video, axis=0)])
    
fnn_train["sarcasm"] = fnn_train["sarcasm"].astype('int').to_numpy()
fnn_test["sarcasm"] = fnn_test["sarcasm"].astype('int').to_numpy()

Train:  (961, 3) (961,) Test:  ((241, 3), (241,))


### Speaker Independent

In [12]:
fnn_train_tensor = FNNTensorDataset(fnn_train[['averaged_context_video_feature', 'averaged_video_feature', 'sarcasm']], False)
fnn_test_tensor = FNNTensorDataset(fnn_test[['averaged_context_video_feature', 'averaged_video_feature', 'sarcasm']], False)

num_of_workers = 0
batch_size = 100
valid_size = 0.2

train_indices = list(range(len(fnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(fnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    fnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    fnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epochs = torch.utils.data.DataLoader(
    fnn_test_tensor, batch_size=fnn_test_tensor.__len__())

In [13]:
model = FNNNetSID()
print(model)

FNNNetSID(
  (fc1): Linear(in_features=4096, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(100):
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()*inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
    if(epoch%10 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = predict_fnn(model, test_loader_epochs)
        print(classification_report(actuals[0].tolist(), predictions[0].tolist(), digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.5000    0.0167    0.0323       120
           1     0.5021    0.9835    0.6648       121

    accuracy                         0.5021       241
   macro avg     0.5011    0.5001    0.3485       241
weighted avg     0.5011    0.5021    0.3498       241

Epoch: 10
              precision    recall  f1-score   support

           0     0.5000    0.0167    0.0323       120
           1     0.5021    0.9835    0.6648       121

    accuracy                         0.5021       241
   macro avg     0.5011    0.5001    0.3485       241
weighted avg     0.5011    0.5021    0.3498       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.5000    0.0167    0.0323       120
           1     0.5021    0.9835    0.6648       121

    accuracy                         0.5021       241
   macro avg     0.5011    0.5001    0.3485       241
weighted avg     0.5011    0.5021    0.3498  

### Speaker Dependent

In [15]:
fnn_train_tensor = FNNTensorDataset(fnn_train[['averaged_context_video_feature', 'averaged_video_feature', 'speaker_encode', 'sarcasm']], True)
fnn_test_tensor = FNNTensorDataset(fnn_test[['averaged_context_video_feature', 'averaged_video_feature', 'speaker_encode', 'sarcasm']], True)

num_of_workers = 0
batch_size = 100
valid_size = 0.2

train_indices = list(range(len(fnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(fnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    fnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    fnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epochs = torch.utils.data.DataLoader(
    fnn_test_tensor, batch_size=fnn_test_tensor.__len__())

In [16]:
model = FNNNetSD()
print(model)

FNNNetSD(
  (fc1): Linear(in_features=4097, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
)


In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(201):
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs, target
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()*inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
    if(epoch%20 == 0):
        print("Epoch: " + str(epoch))
        test_loader_test = torch.utils.data.DataLoader(fnn_test_tensor, batch_size=fnn_test_tensor.__len__())
        predictions, actuals = predict_fnn(model, test_loader_test)
        print(classification_report(actuals[0].tolist(), predictions[0].tolist(), digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.4979    1.0000    0.6648       120
           1     0.0000    0.0000    0.0000       121

    accuracy                         0.4979       241
   macro avg     0.2490    0.5000    0.3324       241
weighted avg     0.2479    0.4979    0.3310       241

Epoch: 20
              precision    recall  f1-score   support

           0     0.5845    0.6917    0.6336       120
           1     0.6263    0.5124    0.5636       121

    accuracy                         0.6017       241
   macro avg     0.6054    0.6020    0.5986       241
weighted avg     0.6055    0.6017    0.5985       241

Epoch: 40
              precision    recall  f1-score   support

           0     0.5874    0.7000    0.6388       120
           1     0.6327    0.5124    0.5662       121

    accuracy                         0.6058       241
   macro avg     0.6100    0.6062    0.6025       241
weighted avg     0.6101    0.6058    0.6023  